In [1]:
import numpy as np
rng = np.random.default_rng(seed = 37)

import matplotlib.pyplot as plt
import os

import torch as t
import torchvision as tv
from torchvision.transforms import v2

from tqdm import tqdm

import sys
sys.path.append('utils')

device = t.device("cuda") if t.cuda.is_available() else t.device("cpu")
print(f'Using {device} for inference')

Using cpu for inference


In [3]:
ENV2 = tv.models.efficientnet_v2_s(weights = tv.models.EfficientNet_V2_S_Weights.DEFAULT).to(device)
ENV2.eval();
for param in ENV2.parameters():
    param.requires_grad = False
inference_transform = tv.models.EfficientNet_B0_Weights.IMAGENET1K_V1.transforms()
inference_transforms = v2.Compose([
    lambda x: x.convert('RGB'),
    inference_transform,
])

In [7]:
fav_pic_index = 28621
data_root = 'imagenet'
imagenet_val_inference = tv.datasets.ImageNet(data_root, split = 'val', transform = inference_transforms)

In [9]:
from distortion import dense_transform_amplitude, sparse_transform_amplitude, create_grid_sample, find_inv_grid, jacobian_det, get_version

In [10]:
def get_sparse_grid_sample(amplitudes, num_diffeo, x_cutoff = 10, y_cutoff = 10, num_terms = 3, x_resolution = 224, y_resolution = 224):
  batch_grid_sample = []
  # batch_inverse_grid_sample = []
  for amp in amplitudes:
    A_nm, B_nm = sparse_transform_amplitude(x_cutoff, y_cutoff, num_terms, amplitude = amp, loop = num_diffeo)
    #A_nm = np.mean(A_nm, axis = 0)
    #B_nm = np.mean(B_nm, axis = 0)
    for A, B in zip(A_nm, B_nm):
      grid_sample = create_grid_sample(x_resolution, y_resolution, A, B)
    #inverse_grid_sample, _ = find_inv_grid(grid_sample)
      batch_grid_sample.append(grid_sample)
    #batch_inverse_grid_sample.append(inverse_grid_sample)
  grid_sample = t.cat(batch_grid_sample)
  return grid_sample

In [13]:
activation = {}
def getActivation(name):
    # the hook signature
    def hook(model, input, output):
        activation[name] = output.detach()
    return hook
handles = []

def retrieve_layer_activation(model, input, layer_index):
  if len(input) == 3: input = input[None, :, :, :]

  layers = list(model.children())
  layers_flat = flatten(layers)

  for index in layer_index:
    handles.append(layers_flat[index - 1].register_forward_hook(getActivation(str(index))))

  with t.no_grad(): model(input)
  for handle in handles: handle.remove()

  return

def flatten(array):
    result = []
    for element in array:
        if hasattr(element, "__iter__"):
            result.extend(flatten(element))
        else:
            result.append(element)
    return result

In [11]:
amp_steps = 50
diffeo_num = 50
diffeo_amp = t.linspace(0, 1.5, amp_steps)
grid_sample = get_sparse_grid_sample(diffeo_amp, diffeo_num, x_cutoff= 10, y_cutoff = 10, num_terms = 3, x_resolution =224, y_resolution= 224)

In [12]:
grid_sample.shape

torch.Size([2500, 224, 224, 2])

In [ ]:
#path = '/content/drive/Shareddrives/Diffeomorphisms_&_CNNs/Alex/data/'
#t.save(grid_sample, path + '20-50-10-10-3-224-224_grid_sample.pt')
last_layer_activation = []
imagenet_val_loader = iter(t.utils.data.DataLoader(imagenet_val_inference, batch_size = 1, shuffle=False))
for i in tqdm(range(100)):
  #file_prefix = f'20-50-10-10-3-224-224_image-{i:04d}_activation'
  val_image, _ = next(imagenet_val_loader)
  distorted_list = t.nn.functional.grid_sample(val_image.repeat(amp_steps * diffeo_num,1,1,1), grid_sample, mode = 'bilinear').to(device)
  retrieve_layer_activation(ENV2, distorted_list, [49])
  last_layer_activation.append(activation['49'])
  activation = {}
  handle = []
